여기서 보자!! 음 eos 토큰은 안쓰이는 숫자 여야 한다.


concepts에 3~126이라 할때 
response가 0,1이고 

concepts도 eos가, response에도 eos가 있어야 하겠지?
concepts+123*response = 이렇게 되면 기존에는 0~122 까지의 값에 0,123 이 더해지는 것이었잖아

오답일 때 가질 수 있는 값은 0~122 
정답일 떄 가질 수 있는 값은 123 ~225


-----------------------------------------------------
concepts을 3~125까지
response를 0,1 이라 할때
concepts+123*response =

Interaction:
오답일 때 가질 수 있는 값은 3~125 
정답일 떄 가질 수 있는 값은 126 ~228

Question:
3~125

;;;;;;;;;;;;
eos =2 일때

Interaction:
오답일 때 가질 수 있는 값은 3~125 + eos 토큰끼리 더해졌을 때 4 추가 --> 곂침
정답일 떄 가질 수 있는 값은 126 ~228 +eos 토큰끼리 더해졌을 때 4

--> 이 경우 r에 더해진 eos값이 123이 곱해지며 또 달리 질 수 잇기에 r값에 eos를 더해주기 전에 미리 123(concept의)숫 자를 구한것을 따로 곱해두고 거기에 eos를 추가하자 
r_shift경우 r을 두개로 만들어서 하나는 interaction용으로 빼두고 하나는 y값으로 쓰이게 eos만 추가해서 따로 두자!

->거기다가 c,r에는 eos/2 를 더해 두개가 더해졌을 때 eos가 되도록 하고 
c_shift, r_shift에는 온전한 eos를 더해두는 거지 그러면 문제 해결?

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import copy
import itertools
import tensorflow as tf
from preprocess.data_utils import read_data,get_max_concepts,calStatistics,extend_multi_concepts,id_mapping,save_id2idx,train_test_split,save_dcur,get_evalmask_token, get_mask_tokens
import pickle
import glob
import matplotlib.pyplot as plt
import time
from models.model_for_kt import TFTransfoXLModel,TFTransfoXLLMHeadModel,TFTransfoXLMLMHeadModel

from transformers import TransfoXLConfig
from tensorflow.keras.utils import register_keras_serializable
import datetime
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"



2024-01-17 13:46:15.114293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-17 13:46:15.698336: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-01-17 13:46:15.698386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-01-17 13:46:15.698391: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:

config_xl = TransfoXLConfig(
    data = '/home/jun/workspace/KT/data/ednet/data.txt',
    dataset = 'wt103',
    d_embed=128,
    d_head = 32,
    d_model=128,
    mem_len=200,
    n_head=8,
    n_layer=6,
    batch_size = 65,
    tgt_len = 140,
    ext_len = 0,
    eval_tgt_len = 36,
    eos_token=2,
    num_c=123,
    mask_token=3,
    C_vocab_size=188,
    Q_vocab_size = 12102,
    R_vocab_size = 2,
    epoch =1
)

In [3]:





# ALL_KEYS = ["fold", "uid", "questions", "concepts", "responses", "timestamps",
#             "usetimes", "selectmasks", "is_repeat", "qidxs", "rest", "orirow", "cidxs"]
# ONE_KEYS = ["fold", "uid"]

total_df, effective_keys = read_data('/home/jun/workspace/KT/data/ednet/data.txt')

stares = []

if 'concepts' in effective_keys:
    max_concepts = get_max_concepts(total_df)
else:
    max_concepts = -1

oris, _, qs, cs, seqnum = calStatistics(total_df, stares, "original") # Check data status

print("="*20)
print(
    f"original total interactions: {oris}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")


# questions ,concepts 값들의 숫자를 재정의 하여 0~ 나오도록 만들 면서 is_repeat값 처리
# 14_54_14 으로 된 concepts를 14,54,14 로 분리하고 기존의 response도 똑같이 확장처리
total_df_ex, effective_keys = extend_multi_concepts(total_df, effective_keys)
total_df, dkeyid2idx = id_mapping(total_df_ex)
dkeyid2idx["max_concepts"] = max_concepts

extends, _, qs, cs, seqnum = calStatistics(
    total_df, stares, "extend multi")
print("="*20)
print(
    f"after extend multi, total interactions: {extends}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")


#train test 분리
train_df, test_df = train_test_split(total_df, 0.1)

#transformer_xl dataset 만들기
train = {"qseqs": [], "cseqs": [], "masked_R": [], "labels": []} #"q_shift": [], "c_shift": [], "r_shift": []}
test = {"qseqs": [], "cseqs": [], "masked_R": [], "labels": []} #"q_shift": [], "c_shift": [], "r_shift": []}


delete bad stu num of len: 7, delete interactions: 11, of r: 0, good num: 6162174
original total interactions: 6162174, qs: 12102, cs: 188, seqnum: 4990
df.columns: Index(['uid', 'is_repeat', 'concepts', 'usetimes', 'timestamps', 'responses',
       'questions'],
      dtype='object')
after extend multi, total interactions: 15220330, qs: 12102, cs: 188, seqnum: 4990
total num: 4990, train+valid num: 4491, test num: 499


In [4]:

# eos_token= 2 # config 파일로 조정할 수 있도록 수정 할 것!
# num_c =123 # config 파일로 조정할 수 있도록 수정 할 것!
for i, row in train_df.iterrows():
#use kc_id or question_id as input

    cseq_list=[int(_) for _ in row["concepts"].split(",")]
    cseq_list.append(config_xl.eos_token)
    train["cseqs"].append(cseq_list)

    qes_list=[int(_) for _ in row["questions"].split(",")]
    qes_list.append(config_xl.eos_token)
    train["qseqs"].append(qes_list)


    rseq_list=[(int(_)) for _ in row["responses"].split(",")]
    rseq_list.append(config_xl.eos_token)
    masked_R, labels = get_mask_tokens(rseq_list,config_xl.mask_token)
    train["masked_R"].append(masked_R)
    train["labels"].append(labels)

for i, row in test_df.iterrows():
    #use kc_id or question_id as input
    cseq_list=[int(_) for _ in row["concepts"].split(",")]
    cseq_list.append(config_xl.eos_token)   
    test["cseqs"].append(cseq_list)

    qes_list=[int(_) for _ in row["questions"].split(",")]
    qes_list.append(config_xl.eos_token)
    test["qseqs"].append(qes_list)


    rseq_list=[(int(_)) for _ in row["responses"].split(",")]
    rseq_list.append(config_xl.eos_token)
    masked_R, labels = get_evalmask_token(rseq_list, config_xl.mask_token, config_xl.eos_token)
    
    test["masked_R"].append(masked_R)
    test["labels"].append(labels)

2024-01-16 18:29:56.980363: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-16 18:29:56.980690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-16 18:29:56.986097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-16 18:29:56.986402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-16 18:29:56.986690: I tensorflow/compiler/xla/stream_executo

TypeError: Cannot convert 0.0 to EagerTensor of dtype int32

In [ ]:
#train
cseqs_list = list(itertools.chain(*train['cseqs']))
qseqs_list = list(itertools.chain(*train['qseqs']))
r_masked_list = tf.concat(train['masked_R'], axis=0)
labels = tf.concat(train['labels'], axis=0)

n_step = len(cseqs_list) // (config_xl.batch_size*config_xl.tgt_len)


sliced_cseqs = tf.slice(cseqs_list,[0],[n_step * config_xl.batch_size*config_xl.tgt_len])  
sliced_qseqs = tf.slice(qseqs_list,[0],[n_step * config_xl.batch_size*config_xl.tgt_len])  
sliced_r_mask = tf.slice(r_masked_list,[0],[n_step * config_xl.batch_size*config_xl.tgt_len]) 
sliced_labels = tf.slice(labels,[0],[n_step * config_xl.batch_size*config_xl.tgt_len]) 

count =len(sliced_cseqs)// (config_xl.batch_size*config_xl.tgt_len)

new_shape = (config_xl.batch_size, -1)  # 나머지 차원은 자동으로 계산됨

cseq_reshaped = tf.reshape(sliced_cseqs, new_shape)
qseq_reshaped = tf.reshape(sliced_qseqs, new_shape)
r_mask_reshaped = tf.reshape(sliced_r_mask, new_shape)
labels_reshaped = tf.reshape(sliced_labels, new_shape)

cseq_transposed = tf.transpose(cseq_reshaped)
qseq_transposed = tf.transpose(qseq_reshaped)
r_mask_transposed = tf.transpose(r_mask_reshaped)
labels_transposed = tf.transpose(labels_reshaped)


#test
test_cseqs_list = list(itertools.chain(*test['cseqs']))
test_qseqs_list = list(itertools.chain(*test['qseqs']))
test_r_masked_list = tf.concat(test['masked_R'], axis=0)
test_labels = tf.concat(test['labels'], axis=0)



# Work out how cleanly we can divide the dataset into bsz parts.
# 아래의 두 코드는   data 텐서에서 배치 크기 bsz로 깔끔하게 맞지 않는 추가 요소를 제거하는 것 배치에 띡 떨어지게
    
test_n_step = len(test_cseqs_list) // (config_xl.batch_size*config_xl.tgt_len)
print('n_step',test_n_step) # 

test_sliced_cseqs = tf.slice(test_cseqs_list,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len])  
test_sliced_qseqs = tf.slice(test_qseqs_list,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len])  
test_sliced_r_masked_seq = tf.slice(test_r_masked_list,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len]) 

test_sliced_labels = tf.slice(test_labels,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len])  

count =len(test_sliced_cseqs)// (config_xl.batch_size*config_xl.tgt_len)
print(count)

'''# 시작 위치와 슬라이싱할 크기 설정
begin = [0]  # 첫 번째 차원의 시작 위치는 0
size = [6]   # 첫 번째 차원에서 6개의 원소를 슬라이싱

# 데이터를 잘라내기 (tf.slice 사용)
sliced_data = tf.slice(data, begin, size)  '''

# Evenly divide the da
# ta across the bsz batches.

new_shape = (config_xl.batch_size, -1)  # 나머지 차원은 자동으로 계산됨

test_qseq_reshaped = tf.reshape(test_sliced_qseqs, new_shape)
test_cseq_reshaped = tf.reshape(test_sliced_cseqs, new_shape)
test_r_masked_seq_reshaped = tf.reshape(test_sliced_r_masked_seq, new_shape)
test_labels_reshaped = tf.reshape(test_sliced_labels, new_shape)


test_qseq_reshaped = tf.transpose(test_qseq_reshaped)
test_cseq_reshaped = tf.transpose(test_cseq_reshaped)
test_r_masked_seq_reshaped = tf.transpose(test_r_masked_seq_reshaped)
test_labels_reshaped = tf.cast(tf.transpose(test_labels_reshaped),tf.int64)


n_step 342
342


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (qseq_transposed, r_mask_transposed, labels_transposed))
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_qseq_reshaped, test_r_masked_seq_reshaped, test_labels_reshaped))

In [ ]:
train_dataset =train_dataset.batch(config_xl.tgt_len)
test_dataset =test_dataset.batch(config_xl.tgt_len)

NameError: name 'train_dataset' is not defined

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = '/home/jun/workspace/KT/logs/gradient_tape/' + current_time +'{}ep_{}mem_Question/train'.format(config_xl.epoch, config_xl.mem_len)
test_log_dir = '/home/jun/workspace/KT/logs/gradient_tape/' + current_time +'{}ep_{}mem_Question/test'.format(config_xl.epoch, config_xl.mem_len)
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
@register_keras_serializable()
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        self.warmup_steps = tf.cast(warmup_steps,tf.float32)

    def __call__(self, step):
        step =tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
    
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
    def get_config(self):
        return {
            'd_model': self.d_model,
            'warmup_steps': self.warmup_steps
            }
learning_rate = CustomSchedule(config_xl.d_model)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


In [ ]:
model = TFTransfoXLMLMHeadModel(config=config_xl)


In [ ]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
  name='test_accuracy')
test_precision = tf.metrics.Precision()
test_recall = tf.metrics.Recall()
train_auc = tf.keras.metrics.AUC()
test_auc = tf.keras.metrics.AUC()


In [ ]:
# Set up logging. 모델그레프 그리기
from datetime import datetime

stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = '/home/jun/workspace/KT/logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)



In [ ]:


@tf.function
def train_step(model, data1,data2, target, mems, optimizer):
    with tf.GradientTape() as tape:
        outputs = model(concepts=data1,responses=data2, labels=target, mems=mems)
        loss = outputs.loss
        mems = outputs.mems
        loss_mx = target != -100
        loss_value = loss[loss_mx]
        loss_value = tf.reshape(loss_value, [-1, config_xl.R_vocab_size])
        labels = target[loss_mx]

        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=loss_value)
        # batch_loss = tf.reduce_sum(loss) / valid_samples
        mean_loss = tf.reduce_mean(loss)
        train_loss(loss)
        train_accuracy(labels,loss_value)
        predictions =tf.nn.softmax(loss_value)
        train_auc(tf.one_hot(labels, depth=predictions.shape[1]), predictions)

    gradients = tape.gradient(mean_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    

    return mems,mean_loss

# def grad(model, inputs, targets):
#   with tf.GradientTape() as tape:
#     loss_value = loss(model, inputs, targets, training=True)
#   return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
def evaluate(model, mems, test_dataset):
    total_loss = 0.0
    num_batches = 0
    evaluation_metrics = []
    

    for test_data1, test_data2, test_target in test_dataset:
        outputs = model(concepts=test_data1, responses=test_data2, labels=test_target, mems=mems, training=False)
        loss = outputs.loss
        mems = outputs.mems

        loss_mx = test_target != -100
        loss_value = loss[loss_mx]
        loss_value = tf.reshape(loss_value, [-1, config_xl.R_vocab_size])
        labels = test_target[loss_mx]

        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=loss_value)
        mean_loss = tf.reduce_mean(loss)

        # Update precision and recall metrics
        predicted_labels = tf.argmax(loss_value, axis=1)
        predictions =tf.nn.softmax(loss_value)

        
        test_auc(tf.one_hot(labels, depth=predictions.shape[1]), predictions)
        test_precision(labels, predicted_labels)
        test_recall(labels, predicted_labels)

        test_accuracy(labels, loss_value)
        test_loss(loss)
        
        
        precision = test_precision.result().numpy()
        recall = test_recall.result().numpy()
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)

        evaluation_metrics.append(test_accuracy.result().numpy())

        total_loss += mean_loss.numpy()
        num_batches += 1

        
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=num_batches)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=num_batches)
            tf.summary.scalar('precision', test_precision.result(), step=num_batches)
            tf.summary.scalar('recall', test_recall.result(), step=num_batches)
            tf.summary.scalar('f1_score', f1_score, step=num_batches)
            tf.summary.scalar('auc', test_auc.result(), step=num_batches)

    # 평균 정밀도, 재현율, F1 점수를 계산합니다.
    average_precision = test_precision.result().numpy()
    average_recall = test_recall.result().numpy()
    average_f1_score = 2 * (average_precision * average_recall) / (average_precision + average_recall + 1e-7)

    average_metric = sum(evaluation_metrics) / len(evaluation_metrics)
    average_loss = total_loss / num_batches

    return average_loss, average_metric, average_precision, average_recall, average_f1_score


In [ ]:
loss_values = []
num_batches = 0
# tf.summary.trace_on(graph=True, profiler=True)

for epoch in range(config_xl.epoch):
    start = time.time()
    total_loss = 0.0
    mems = None              # 첫 번째 모델의 메모리 상태
           
    
    for data1, data2, target in test_dataset:
    # for data1, data2, target in train_dataset:
        mems, loss_value = train_step(model, data1,data2, target, mems, optimizer)
        
        num_batches += 1
        total_loss += loss_value.numpy()
        if num_batches % 100 == 0:
            loss_values.append(loss_value.numpy())
            print(f'Epoch {epoch + 1} Batch {num_batches} Loss {loss_value.numpy()}')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=num_batches)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=num_batches)
            tf.summary.scalar('auc', train_auc.result(), step=num_batches)

    # with writer.as_default():# 텐서보드에 모델 그래프 그리기
    #     tf.summary.trace_export(
    #         name="my_func_trace",
    #         step=0,
    #         profiler_outdir=logdir)             

    # Reset metrics every epoch
    # train_loss.reset_states()
    # test_loss.reset_states()
    # train_accuracy.reset_states()
    # test_accuracy.reset_states()

2024-01-16 15:52:13.740674: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x564760618790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-16 15:52:13.740694: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2024-01-16 15:52:13.740698: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2024-01-16 15:52:13.744377: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-16 15:52:13.823347: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1 Batch 100 Loss 0.6370684504508972
Epoch 1 Batch 200 Loss 0.58848637342453
Epoch 1 Batch 300 Loss 0.5493931174278259
Epoch 1 Batch 400 Loss 0.4001980125904083
Epoch 1 Batch 500 Loss 0.31552058458328247
Epoch 1 Batch 600 Loss 0.2752041518688202
Epoch 1 Batch 700 Loss 0.2523006498813629
Epoch 1 Batch 800 Loss 0.1527223289012909
Epoch 1 Batch 900 Loss 0.15761373937129974
Epoch 1 Batch 1000 Loss 0.14871670305728912
Epoch 1 Batch 1100 Loss 0.14128609001636505
Epoch 1 Batch 1200 Loss 0.13805948197841644
Epoch 1 Batch 1300 Loss 0.11757903546094894
Epoch 2 Batch 1400 Loss 0.15114034712314606
Epoch 2 Batch 1500 Loss 0.12192797660827637
Epoch 2 Batch 1600 Loss 0.16158121824264526
Epoch 2 Batch 1700 Loss 0.11922312527894974
Epoch 2 Batch 1800 Loss 0.1304556429386139
Epoch 2 Batch 1900 Loss 0.10752145946025848
Epoch 2 Batch 2000 Loss 0.13480199873447418
Epoch 2 Batch 2100 Loss 0.12593580782413483
Epoch 2 Batch 2200 Loss 0.12249918282032013
Epoch 2 Batch 2300 Loss 0.12609970569610596
Epoch 2

In [18]:
# test_mems = None
# test_loss0,test_acc0,average_precision, average_recall, average_f1_score = evaluate(model,test_mems, test_dataset)
# print(f'Test Loss on First Half Dataset after Epoch {epoch + 1}: {test_loss0}')

Test Loss on First Half Dataset after Epoch 5: 0.6319265374314715


In [ ]:
# model.save_weights('/home/jun/workspace/KT/save_model/5ep_600mem_questiion.ckpt/my_checkpoint') 
model.save('/home/jun/workspace/KT/save_model/5ep_600')

# model.load_weights('/home/jun/workspace/KT/save_model/5ep_600mem.ckpt/my_checkpoint')



In [20]:
print(test_loss0)
print(test_acc0)
print(average_f1_score)


0.6319265374314715
0.6484004206824721
0.7520016045546908


###임베딩 데이터 시각화

In [21]:
from tensorboard.plugins import projector

log_dir='/home/jun/workspace/KT/logs/question/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [22]:
# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for concepts in dkeyid2idx['questions']:
    f.write("{}\n".format(concepts))
 

In [23]:
weights = tf.Variable(model.transformer.word_emb_C.get_weights()[0])


In [24]:
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'/home/jun/workspace/KT/logs/question/embedding.ckpt-1'

In [25]:
# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [26]:
#tensorboard --logdir /logs/imdb-example/  실행코드
#체크포인트 저장하는 방법 공부해보자!

In [5]:
import os

# Step 1: Get the folder path from the user
folder_path = '/home/jun/workspace/KT/data/ednet/KT12/KT1/'

# Step 2: Initialize a variable to store the count of files
file_count = 0

# Step 3: Iterate through the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is not a CSV file
    if os.path.isfile(os.path.join(folder_path, filename)) and not filename.endswith('.csv'):
        file_count += 1

# Step 4: Print the number of non-CSV files in the folder
print(f"Number of non-CSV files in the folder: {file_count}")

Number of non-CSV files in the folder: 0
